In [8]:

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import pickle
import re
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import dash_bootstrap_components

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio


In [7]:

#load the tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

#load the encoder
with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# import the model
model = load_model('cnn_lstm_model.h5')


# Define the ArabicCleaning class for text preprocessing
class ArabicCleaning():
    def __init__(self):
        pass

    def clean(self, text):
        arabic_pattern = r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]'
        arabic_pattern_others = r'[^\w\s\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]'

        def remove_special_words(text):
            words = text.split()
            text = [word for word in words if '#' not in word and '_' not in word]
            text = ' '.join(text)
            return text

        def keep_only_arabic_letters(text):
            words = text.split()
            processed_words = []
            for word in words:
                arabic_letters_only = ''.join([char for char in word if re.match(arabic_pattern, char) and char not in ["؟", "؛", "،"]])
                processed_words.append(arabic_letters_only)
            return ' '.join(processed_words)

        def check_empty(text):
            if len(text.split()) == 0:
                return ''
            else:
                return text

        text = remove_special_words(text)
        text = re.sub(arabic_pattern_others, '', text)
        text = re.sub(r'[0-9]', '', text)
        text = re.sub(r'[a-zA-Z]', '', text)
        text = keep_only_arabic_letters(text)
        text = check_empty(text)

        return text
    
    
# Define the function to preprocess and predict

def preprocess_and_predict(text, tokenizer, model, encoder):
    # Clean and preprocess the text
    clean_text = ArabicCleaning().clean(text)
    sequences = tokenizer.texts_to_sequences([clean_text])
    padded_sequences = pad_sequences(sequences, maxlen=65)
    
    # Predict the probabilities
    prediction = model.predict(padded_sequences)
    
    # Get the class names and their corresponding probabilities
    class_names = encoder.classes_
    probabilities = prediction[0]
    dialect = encoder.inverse_transform([np.argmax(prediction)])
    # creat a dictionary with the class names and their corresponding probabilities
    countries = {"EG":"Egypt","LB":"Lebanon","LY":"Libya","MA":"Morocco","SD":"Sudan"}
    probabilities_dict = dict(zip(class_names, probabilities))
    # Replace country codes with their corresponding names
    probabilities_dict = {countries.get(key, key): value for key, value in probabilities_dict.items()}
    return class_names, probabilities , countries[dialect[0]] , probabilities_dict


# Define the function to preprocess and predict

def preprocess_and_predict(text, tokenizer, model, encoder):
    # Clean and preprocess the text
    clean_text = ArabicCleaning().clean(text)
    sequences = tokenizer.texts_to_sequences([clean_text])
    padded_sequences = pad_sequences(sequences, maxlen=65)
    
    # Predict the probabilities
    prediction = model.predict(padded_sequences)
    
    # Get the class names and their corresponding probabilities
    class_names = encoder.classes_
    probabilities = prediction[0]
    dialect = encoder.inverse_transform([np.argmax(prediction)])
    # creat a dictionary with the class names and their corresponding probabilities
    countries = {"EG":"Egypt","LB":"Lebanon","LY":"Libya","MA":"Morocco","SD":"Sudan"}
    probabilities_dict = dict(zip(class_names, probabilities))
    # Replace country codes with their corresponding names
    probabilities_dict = {countries.get(key, key): value for key, value in probabilities_dict.items()}
    return class_names, probabilities , countries[dialect[0]] , probabilities_dict


# Define the function to plot the probabilities

def plot_probabilities(class_names, probabilities):
    # Sort the probabilities and class names in descending order
    sorted_indices = np.argsort(probabilities)
    sorted_probabilities = np.array(probabilities)[sorted_indices]
    sorted_class_names = np.array(class_names)[sorted_indices]

    # Identify the index of the maximum probability (which will be 0 after sorting)
    max_index = 4

    # Create the horizontal bar plot with text annotations
    fig = go.Figure(go.Bar(
        x=sorted_probabilities,
        y=sorted_class_names,
        orientation='h',
        marker=dict(
            color=sorted_probabilities,
            colorscale='Viridis_r'  # Use Viridis_r color scale
        ),
        text=[f'{p*100:.2f}%' for p in sorted_probabilities],  # Add percentages
        textposition='inside',  # Position the text inside the bars
        insidetextanchor='middle',  # Anchor the text in the middle
    ))

    # Customize the layout
    fig.update_layout(
        title='Probabilities of Each Dialect',
        xaxis_title='Probability',
        yaxis_title='Dialect',
        width=600,  # Adjust the width to make it more square
        height=600  # Adjust the height to make it more square
    )

    # Show the plot
    pio.show(fig)
    return fig

# Define the function to plot the world map
def world_partic(probabilities_dict):
    

    data = {
        'country': probabilities_dict.keys(),
        'value': probabilities_dict.values()
    }
    df = pd.DataFrame(data)

    fig = px.choropleth(
        df,
        locations="country",
        color="value",
        locationmode="country names",
        scope="world",  # Use world scope to display both continents
        title="Probabilities of Each Dialect",
        color_continuous_scale=px.colors.sequential.Viridis_r  # Blue for smaller, Red for larger
    )


    # Update layout to focus on the region of interest (Africa and Asia)
    fig.update_geos(
        projection_type="natural earth",
        showcountries=True,
        lataxis_range=[0, 40],  # Latitude range to cover Africa and Asia
        lonaxis_range=[-20, 45]  # Longitude range to cover Africa and Asia
    )


    for country, value in probabilities_dict.items():
        fig.add_scattergeo(
            locations=[country],
            locationmode="country names",
            text=[country],
            mode='text',
            textfont=dict(
                color='black',
                family='Arial',
                size=15,


            ),
            showlegend=False
        )
    fig.update_layout(
        title='Probabilities of Each Dialect',
        xaxis_title='Probability',
        yaxis_title='Dialect',
        width=600,  # Adjust the width to make it more square
        height=600  # Adjust the height to make it more square
    )
    # show the fig
    pio.show(fig)
    return fig
    



In [9]:

# text = "ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص"
# class_names, probabilities , dialect ,probabilities_dict= preprocess_and_predict(text, tokenizer, model, encoder)
# plot_probabilities(class_names, probabilities)
# world_partic(probabilities_dict)


In [10]:


# Define your Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

# Define the layout of the welcome page
welcome_layout = html.Div([
    html.H1('Welcome to Dialect Prediction App'),
    html.P('Click the button below to navigate to the data page.'),
    dcc.Link('Go to Data Page', href='/data', className='btn btn-primary')
])

# Define the layout of the data page
data_layout = dbc.Container([
    html.H1('Dialect Prediction Data', className='mb-4'),
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Label('Enter text:'),
                dcc.Textarea(id='text-input', placeholder='Enter text...', className='form-control mb-4')
            ]),
            html.Div(id='prediction-output', className='mb-4'),
            html.Button('Generate Data', id='generate-button', n_clicks=0, className='btn btn-primary')
        ], md=6),
        dbc.Col([
            html.Div(id='probabilities-plot-container'),
            html.Div(id='world-partic-plot-container')
        ], md=6)
    ])
], className='mt-5')

# Define callbacks to handle interactions
@app.callback(
    Output('prediction-output', 'children'),
    Output('probabilities-plot-container', 'children'),
    Output('world-partic-plot-container', 'children'),
    Input('generate-button', 'n_clicks'),
    State('text-input', 'value')
)
def update_output(n_clicks, text):
    if n_clicks and text:
        class_names, probabilities, dialect, probabilities_dict = preprocess_and_predict(text, tokenizer, model, encoder)

        prediction_output = html.Div([
            html.Label('The predicted Dialect Origin'),
            html.Div(dialect)
        ])

        probabilities_plot = plot_probabilities(class_names, probabilities)
        world_partic_plot = world_partic(probabilities_dict)

        return prediction_output, dcc.Graph(figure=probabilities_plot), dcc.Graph(figure=world_partic_plot)

    return '', None, None

# Define the app layout
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Define callback to switch pages
@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname')
)
def display_page(pathname):
    if pathname == '/data':
        return data_layout
    else:
        return welcome_layout

# Run the- app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
## Morocoo
# data = {'text': '''
        
#         مابغيتش نشوفك مرة أخرى 
# كنتمنى كتكون هادي كاتضحك 
# عندي واحد السر بغا نقول ليك
#  واش عندك الوقت باش تهضر معايا
#  سمح ليا داكشي علاش ماكنشوفوش وقت طويل        

#         '''}


# libya
# data = {'text': '''
        
# ايه حتى احنى عندنا صبي بالضمة تعني اسكبي لاكن صبي بالفتحة تعني انهضي
# في الشرق الليبي انقولوا سمحة اي جميلة مثلا  سلوى سمحة تعني سلوى جميلة
# في الشرق الليبي  انقولوا  منوا  مثلا منوا ايدق الباب  بمعنى من يطرق الباب وكذلك نقول منهو اللي ادق الباب
# باهي اكويس الحمدلله
       
#         '''}


#suddan
# data = {'text': '''
        
# االمال تلتوا ولا كتلتوا.
# بادر الضيف ولو ببرش وقرعة موية
# مسيك دربين ضهيب وركيب سرجين وقيع
# ذاد الحِبان ليهو مكان  أو ذاد الحِبان له مكان
        
#         '''}


# #Lebanon
# data = {'text': '''
        
# يلّي باعك ببصلة، بيعو بقشرتها! يلّي بيعملك حمار، البُطة! إذا قلتلَّك تقبرني بتجيب المجرفة وبتطمني؟ الأكل ما إلك، بطنك ما إلك؟
        
#         '''}


# # Egpyt
# data = {'text': '''
        
# لو بتحبها كلم الى جمبها
# عمر الجزيرة ما تروح لشبه الجزيرة
#  بس أنت وعدتني إنكّ عمرك ما هتسبني؟!
# = مّنا عملتهالك مُفاجأه.
# الرجولة ضرب نار مش اكل فيشار
# ياللي خايف من خليك احسن لك مع العيال الطرية
# بدل ما تبصلها هات جردل واغسلها
# يلى عينك علينا شيلها الله يهديك ويهدينا
# مبلاش تقرب منى الكل زعل منى
# متلمسهاش دى جاية بعرقى مش ببلاش      
          
#         '''}